In [1]:
import numpy as np
import pandas as pd
from os import path
from collections import defaultdict
from scipy.io import loadmat
from nilearn.glm.first_level import FirstLevelModel

# imports for data loading, processing and analysis
from utils.io import get_args, get_firstlevel_dir, save_args, load_data
from utils.preprocessing import set_get_timeseries, build_sub_run_df, convert_df_to_desMat, add_VTC_get_breaks, exclude_irrelevant_VTC_TRs, check_dataset2_censored_trs
from utils.nilearn_analysis import get_contrast_info, fit_edge_flm, get_flm_attribute

/opt/homebrew/Caskroom/miniforge/base/envs/accel/lib/python3.10/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '


In [2]:
# Set rng for replication
np.random.seed(2022)

In [3]:
args = get_args(['-dataset', '2'])
args.model = 'VTC_shifted' # hard coding to make sure VTC shift and break clipping occurs

# Set up directories, load data
results_dir = get_firstlevel_dir(args)
save_args(args, results_dir)

ridx, cidx = np.tril_indices(args.nROI, -1)
get_timeseries = set_get_timeseries(args, ridx, cidx)  # set the type of timeseries to use, ROI or edge

contrasts, contrast_cols = get_contrast_info(args)

timeseries_data, sub_file_map, sublist, ntr_tossed = load_data(args)

In [4]:
# PREPROCESS BEHAV DATA, MAKE DESIGN MATRICES
des_mat_dict = defaultdict(list)
fitable_timeseries = defaultdict(list)
for sidx, sub in enumerate(sublist):
    sub_files = sub_file_map[sidx]

    for runidx, sub_file in enumerate(sub_files):
        sub_trs = np.arange(timeseries_data[sidx][runidx].shape[0])*args.t_r
        
        sub_run_df = build_sub_run_df(loadmat(sub_file), ntr_tossed, args)
        
        sub_run_df, (break_onsets, break_offsets, break_durations) = add_VTC_get_breaks(sub_run_df, args)

        sub_run_desmat = convert_df_to_desMat(
                sub_run_df,
                sub_trs,
                model='VTC_shifted',
                VTC_shift = args.VTC_shift,
                break_durations=break_durations,
            )

        # Include a session if a subject experienced all trial types for the contrast
        if all(ev in sub_run_desmat.columns for ev in contrast_cols):
            curr_timeseries = get_timeseries(timeseries_data[sidx][runidx])
            
            curr_timeseries, sub_run_desmat, skip_session, bad_trs = check_dataset2_censored_trs(sub_run_desmat, curr_timeseries, args, return_bad_trs=True)
            
            # GET RESIDUALS FROM EVENT GLM
            if not skip_session:
                sub_run_events_desmat = convert_df_to_desMat(
                    sub_run_df,
                    sub_trs,
                    model='trialType',
                    VTC_shift = args.VTC_shift,
                    break_durations=break_durations,
                )
                if (int(args.dataset)==2) and (args.glm_noise_model=='ar1'):
                    # add a junk regressor for every non-border bad TR
                    for btr in bad_trs:
                        sub_run_events_desmat[f'censor_{btr}'] = 0
                        sub_run_events_desmat.loc[btr, f'censor_{btr}'] = 1
                    # reset desmat index
                    sub_run_events_desmat = sub_run_events_desmat.reset_index(drop=True)
                
                # fit and get residuals
                fitted_flm = fit_edge_flm(
                                FirstLevelModel(t_r=args.t_r, signal_scaling=args.signal_scaling, noise_model=args.glm_noise_model, minimize_memory=False),
                                run_Ys=[curr_timeseries],
                                design_matrices=[sub_run_events_desmat],
                            )
                curr_timeseries = get_flm_attribute(fitted_flm, 'residuals', result_as_time_series=True)[0]

                # shift VTC and drop block breaks
                sub_run_desmat, curr_timeseries = exclude_irrelevant_VTC_TRs(sub_run_desmat, curr_timeseries, args, break_onsets, break_offsets)
                des_mat_dict[sub].append(sub_run_desmat)    
                fitable_timeseries[sub].append(curr_timeseries)

A 'modulation' column was found in the given events data and is used.
A 'modulation' column was found in the given events data and is used.
A 'modulation' column was found in the given events data and is used.
A 'modulation' column was found in the given events data and is used.
A 'modulation' column was found in the given events data and is used.
A 'modulation' column was found in the given events data and is used.
A 'modulation' column was found in the given events data and is used.
A 'modulation' column was found in the given events data and is used.
A 'modulation' column was found in the given events data and is used.
A 'modulation' column was found in the given events data and is used.
A 'modulation' column was found in the given events data and is used.
A 'modulation' column was found in the given events data and is used.
A 'modulation' column was found in the given events data and is used.
A 'modulation' column was found in the given events data and is used.
A 'modulation' colum

In [5]:
def corr2_coeff(A, B):
    # See https://stackoverflow.com/questions/42677677/python3-computationally-efficient-correlation-between-matrix-and-array
    # Rowwise mean of input arrays & subtract from input arrays themeselves
    A_mA = A - A.mean(1)[:, None]
    B_mB = B - B.mean(1)[:, None]

    # Sum of squares across rows
    ssA = (A_mA**2).sum(1)
    ssB = (B_mB**2).sum(1)

    # Finally get corr coeff
    return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))

In [6]:
corr_df = pd.DataFrame()
for sidx, sub in enumerate(des_mat_dict):
    n_runs = len(des_mat_dict[sub])
    if args.replicate:
        sub_zs = np.full((n_runs, args.nROI), np.nan)
    else:    
        sub_zs = np.full((n_runs, len(ridx)), np.nan)
    
    for run_idx in range(n_runs):
        VTC_ts = des_mat_dict[sub][run_idx].vtc.values
        edge_ts = fitable_timeseries[sub][run_idx]
        corrs2 = corr2_coeff(edge_ts.T,VTC_ts[:, np.newaxis].T).ravel()
        zs = np.arctanh(corrs2)
        sub_zs[run_idx, :] = zs
    
    corr_df = pd.concat([corr_df, 
              pd.DataFrame(
                  np.mean(sub_zs, 0)
              ).T
              ])    


In [7]:
# SAVE FIRSTLEVEL RESULTS
results_str = f'model-VTCcorr_contrast-VTCcorr_datatype-{"roi" if args.use_rois else "edge"}.csv'
corr_df.to_csv(path.join(results_dir, results_str), index=False)